# Language Modeling

- A probabilistic technique used to __predict the probability that a sequence of tokens belongs to a language__!
- __Applications__ - when having several options and have to __choose the most likely__, based on the context words:
    - Compare sentences to tell __which are the 'good' ones__.
    - __Spell-checking__ - we receive a user's query 'I want some cugar in my coffee.' -> cougar or sugar?
    - __Automatic Speech recognition__ - get most likely sequence of words.
    - __Machine Translation, Question Answering, Summarization__, etc.
    - __Information retrieval__ - the document is likely to best answer a query if it is most likely to generate the query (it contains the same words as in the query)
- Language models can __score and sort__ sentences. <br>
P(I like apples) >> P(I lick apples) <br>
- Language models encode some __grammaticality__. <br>
P(I was going to call) >> P(I will going to call)

# Basic Probabilistic Approach

P(I, want, some, sugar, in, my, coffee) = ?

## First shot: Chain Rule of probability

$P(w_{1},..,w_{n}) = \prod{P(w_{i}|w_{1},..,w_{i-1})}$

P(I,want,some,cougar,in,my,coffee) = P(want|I)\*P(some|I, want)\*P(cougar|I,want,some)...\*P(coffee|I,want,some,cougar,in,my)

In [1]:
# ! pip3 install tqdm
import math
import collections
import nltk
from tqdm import tqdm
import pandas as pd

In [2]:
emma_words = nltk.corpus.gutenberg.words('austen-emma.txt')
print(emma_words[450:520])
emma_words_joined = " ".join(emma_words)

['event', 'had', 'every', 'promise', 'of', 'happiness', 'for', 'her', 'friend', '.', 'Mr', '.', 'Weston', 'was', 'a', 'man', 'of', 'unexceptionable', 'character', ',', 'easy', 'fortune', ',', 'suitable', 'age', ',', 'and', 'pleasant', 'manners', ';', 'and', 'there', 'was', 'some', 'satisfaction', 'in', 'considering', 'with', 'what', 'self', '-', 'denying', ',', 'generous', 'friendship', 'she', 'had', 'always', 'wished', 'and', 'promoted', 'the', 'match', ';', 'but', 'it', 'was', 'a', 'black', 'morning', "'", 's', 'work', 'for', 'her', '.', 'The', 'want', 'of', 'Miss']


In [3]:
sentences = [['She', 'was', 'the', 'youngest'],
            ['I', 'want', 'some', 'sugar', 'in', 'my', 'coffee'],
            ['she', 'had', 'always', 'wished', 'for', 'this', 'opportunity']]

for sentence in sentences:
    sentence_prob = 1
    for i in range(len(sentence)):
        
        context_freq = emma_words_joined.count(" ".join(sentence[:i+1]))
        print("Occurence of:", sentence[:i+1], context_freq)
        word_context_freq = emma_words_joined.count(" ".join(sentence[:i+2]))
        print("Occurence of:", sentence[:i+2], word_context_freq)
        
        if context_freq == 0 or word_context_freq == 0: continue
        sentence_prob *= word_context_freq / context_freq
        
    print("Sentence probability: {}".format(sentence_prob))

Occurence of: ['She'] 570
Occurence of: ['She', 'was'] 110
Occurence of: ['She', 'was'] 110
Occurence of: ['She', 'was', 'the'] 6
Occurence of: ['She', 'was', 'the'] 6
Occurence of: ['She', 'was', 'the', 'youngest'] 1
Occurence of: ['She', 'was', 'the', 'youngest'] 1
Occurence of: ['She', 'was', 'the', 'youngest'] 1
Sentence probability: 0.0017543859649122805
Occurence of: ['I'] 4016
Occurence of: ['I', 'want'] 18
Occurence of: ['I', 'want'] 18
Occurence of: ['I', 'want', 'some'] 0
Occurence of: ['I', 'want', 'some'] 0
Occurence of: ['I', 'want', 'some', 'sugar'] 0
Occurence of: ['I', 'want', 'some', 'sugar'] 0
Occurence of: ['I', 'want', 'some', 'sugar', 'in'] 0
Occurence of: ['I', 'want', 'some', 'sugar', 'in'] 0
Occurence of: ['I', 'want', 'some', 'sugar', 'in', 'my'] 0
Occurence of: ['I', 'want', 'some', 'sugar', 'in', 'my'] 0
Occurence of: ['I', 'want', 'some', 'sugar', 'in', 'my', 'coffee'] 0
Occurence of: ['I', 'want', 'some', 'sugar', 'in', 'my', 'coffee'] 0
Occurence of: ['I',

__ Problem: __ Will need a __big corpora of text__ in order to have all possible sentences, occurring enough times! But the possible sentences are too much and we will never see enough of them!

## Markov Simplifying Assumption of Independence:
- Take only __l__ of the words before the given one:
$ P(w_1,...,w_n) = \prod{P(w_i|w_{i-l},...,w_{i-1})} $
- __L=1 Unigram Model__:<br>
$ P(w_i|w_{i-l},...,w_{i-1}) = P(w_i) $  <br>
P(coffee|I,want,some,cougar,in,my) ~ P(coffee)
- __L=2 Bigram Model__ : Condition only on the previous word<br>
$ P(w_i|w_{i-l},...,w_{i-1}) = P(w_i|w_{i-1}) $ <br>
P(coffee|I,want,some,cougar,in,my) ~ P(coffee|my)
- If we want to add more context and thus, more dependence b/n words, we can make a tri-gram, a four-gram, etc. model!

## Maximum Likelihood estimation: (the bigram case)
$P(w_i|w_{i-1}) = \frac{c(w_{i-1}, w_i)}{c(w_i)}$

__Exercise__: What is the downside of using an unigram model? <br>
<br>
__Exercise__: Having this __training data__:<br>
[s] I am Sam [/s]<br>
[s] Sam I am [/s]<br>
[s] Sam I like [/s]<br>
[s] Sam I do like [/s]<br>
[s] do I like Sam [/s]<br>

Construct a __bigram model__ and then estimate:<br>
1. What is the __most probable next word__ for the following word sequences:<br>
(1) [s] Sam . . .<br>
(2) [s] Sam I do . . .<br>
(3) [s] Sam I am Sam . . . <br>
(4) [s] do I like . . .<br>

2. Which of the following sentences is __better__:<br>
(5) [s] Sam I do I like [/s]<br>
(6) [s] Sam I am [/s]<br>
(7) [s] I do like Sam I am [/s]<br>

In [4]:
# Compute co-occurence of bi-grams, then normalize by unigram frequency
first_most_freq_words = collections.Counter(emma_words).most_common(200)
print("Most frequent words: ", first_most_freq_words)

unique_words = [_w[0] for _w in first_most_freq_words]
unigram_counts = [emma_words.count(_w) for _w in tqdm(unique_words)]

coocurrence_df = pd.DataFrame({'word': unique_words, 'unigram_count': unigram_counts})
coocurrence_df.head()

  0%|          | 0/200 [00:00<?, ?it/s]

Most frequent words:  [(',', 11454), ('.', 6928), ('to', 5183), ('the', 4844), ('and', 4672), ('of', 4279), ('I', 3178), ('a', 3004), ('was', 2385), ('her', 2381), (';', 2199), ('it', 2128), ('in', 2118), ('not', 2101), ('"', 2004), ('be', 1970), ('she', 1778), ('that', 1730), ('you', 1677), ('had', 1606), ('as', 1387), ('--', 1382), ('he', 1365), ('for', 1321), ('have', 1301), ('is', 1220), ('with', 1187), ('Mr', 1153), ('very', 1151), ('but', 1148), ('."', 1138), ('his', 1088), ("'", 1007), ('at', 997), ('s', 933), ('so', 924), ('Emma', 865), ('all', 835), ('could', 825), ('would', 815), ('been', 759), ('him', 758), ('Mrs', 699), ('.--', 685), ('on', 677), ('any', 651), ('my', 619), ('no', 616), ('Miss', 592), ('were', 591), ('do', 580), ('-', 574), ('must', 564), ('me', 564), ('She', 562), ('will', 559), ('by', 558), ('which', 552), ('!', 549), ('from', 535), ('Harriet', 506), ('or', 490), ('said', 484), ('much', 478), ('more', 464), ('an', 452), ('are', 447), ('He', 441), ('such', 

100%|██████████| 200/200 [00:59<00:00,  3.44it/s]


,unigram_count,word
0,11454,","
1,6928,.
2,5183,to
3,4844,the
4,4672,and


In [5]:
# Let's add bi-gram counts!
for word in tqdm(unique_words):
    coocurrence_df[word] = coocurrence_df.apply(lambda x: emma_words_joined.count(x['word']+" "+word), axis=1)
coocurrence_df.head()

100%|██████████| 200/200 [00:28<00:00,  7.99it/s]


,unigram_count,word,",",.,to,the,and,of,I,a,...,There,enough,mind,No,A,Highbury,does,;--,happy,even
0,11454,",",0,0,289,387,1880,110,570,2547,...,0,1,0,0,3,2,5,0,2,17
1,6928,.,0,20,0,0,52,0,977,54,...,80,0,0,83,292,2,0,0,0,0
2,5183,to,26,21,10,339,4,5,16,356,...,0,0,2,0,6,21,0,0,0,0
3,4844,the,0,2,9,1,0,10,2,179,...,0,0,4,0,15,4,0,0,2,47
4,4672,and,144,33,66,376,6,42,165,397,...,0,0,4,1,2,5,2,1,8,8


In [6]:
# Now, we have to divide each bi-gram count by the count of the unigram to produce the probabilities
def normalize_row(row):
    for _word in unique_words: 
        row[_word] /= row['unigram_count']
    return row

bigrams_probabilites = coocurrence_df.apply(lambda x: normalize_row(x), axis=1)
bigrams_probabilites.head()

,unigram_count,word,",",.,to,the,and,of,I,a,...,There,enough,mind,No,A,Highbury,does,;--,happy,even
0,11454,",",0.000000,0.000000,0.025231,0.033787,0.164135,0.009604,0.049764,0.222368,...,0.000000,0.000087,0.000000,0.000000,0.000262,0.000175,0.000437,0.000000,0.000175,0.001484
1,6928,.,0.000000,0.002887,0.000000,0.000000,0.007506,0.000000,0.141022,0.007794,...,0.011547,0.000000,0.000000,0.011980,0.042148,0.000289,0.000000,0.000000,0.000000,0.000000
2,5183,to,0.005016,0.004052,0.001929,0.065406,0.000772,0.000965,0.003087,0.068686,...,0.000000,0.000000,0.000386,0.000000,0.001158,0.004052,0.000000,0.000000,0.000000,0.000000
3,4844,the,0.000000,0.000413,0.001858,0.000206,0.000000,0.002064,0.000413,0.036953,...,0.000000,0.000000,0.000826,0.000000,0.003097,0.000826,0.000000,0.000000,0.000413,0.009703
4,4672,and,0.030822,0.007063,0.014127,0.080479,0.001284,0.008990,0.035317,0.084974,...,0.000000,0.000000,0.000856,0.000214,0.000428,0.001070,0.000428,0.000214,0.001712,0.001712


In [7]:
# FInally copute probability for some sentence
sentence = ['she', 'was', 'to', 'the']
final_probability = 1

for _w1, _w2 in zip(sentence[:-1], sentence[1:]):
    prob = bigrams_probabilites[bigrams_probabilites['word'] == _w2][_w1]
    if len(prob)>0:
        final_probability *= prob.iloc[0]
        print(_w1, _w2, prob.iloc[0])
    else:
        print(_w1, _w2, 'Not Found')
    
print(final_probability)

she was 0.0029350104821802936
was to 0.00019293845263360988
to the 0.0018579686209744012
1.0521237465023985e-09


## Problem #1
Multiplying individual bigram probabilities we end up with very __small numbers__ and there is a risk of __underflow__. Solution:

__log(p1\*p2\*p3\*p4) = log(p1) + log(p2) + log =(p3) + log(p4)__

In [8]:
final_probability = 0

for _w1, _w2 in zip(sentence[:-1], sentence[1:]):
    prob = bigrams_probabilites[bigrams_probabilites['word'] == _w2][_w1]
    if len(prob)>0:
        final_probability += prob.iloc[0]
    
print(final_probability)

0.004985917555788305


## Problem #2 : We still have to obtain an enourmous corpus in order to prevent having zero counts for some possible bigrams!
- __Google N-grams__:  number of __bigrams: 314,843,401__, number of __tokens: 1,024,908,267,229__ - 6% of all volumes published in English
- __Shakespeare__: __884,647 words, 29,066 tokens__. Produced 300,000 bigram types, out of __844 million possible bigrams__ - small percentage of the bigrams were ever seen.

# Evaluation of a language models
- __Extrinsic__ - with respect to some __task__ - e.g. how well it predicted which is the correct spelling of a word in a spell-checking task.
- __Intrinsic__ - estimate how well the model increases the probability for sentences from a __test corpus__. (! Should have same distribution of words in both sets). 
    - __Shanon game__ - how well will we __predict the next word__, given some context?
    - __Perplexity__: 
$ PP(W) = P(w_1, w_2, ..., w_n)^{-\frac{1}{n}} $
    - For __uniform distribution__, perplexity equals the size of vocabulary:
        - probability for generating 0, 1, 2, 3, 4, 5?
    - __Minimizing perplexity__ is like maximizing probability!

__Exercise__: Compute the __perplexity__ with the same __training data__:  <br>
[s] I am Sam [/s] <br>
[s] Sam I am [/s] <br>
[s] Sam I like [/s] <br>
[s] Sam I do like [/s] <br>
[s] do I like Sam [/s] <br>

for the test __sentence__ : <br>
[s] I do like Sam

# Smoothing Techniques
- Generally, we want to avoid sparse statistics and to decide what to do in case we have a bi-gram count = 0.

In [9]:
bigrams_probabilites.head()

,unigram_count,word,",",.,to,the,and,of,I,a,...,There,enough,mind,No,A,Highbury,does,;--,happy,even
0,11454,",",0.000000,0.000000,0.025231,0.033787,0.164135,0.009604,0.049764,0.222368,...,0.000000,0.000087,0.000000,0.000000,0.000262,0.000175,0.000437,0.000000,0.000175,0.001484
1,6928,.,0.000000,0.002887,0.000000,0.000000,0.007506,0.000000,0.141022,0.007794,...,0.011547,0.000000,0.000000,0.011980,0.042148,0.000289,0.000000,0.000000,0.000000,0.000000
2,5183,to,0.005016,0.004052,0.001929,0.065406,0.000772,0.000965,0.003087,0.068686,...,0.000000,0.000000,0.000386,0.000000,0.001158,0.004052,0.000000,0.000000,0.000000,0.000000
3,4844,the,0.000000,0.000413,0.001858,0.000206,0.000000,0.002064,0.000413,0.036953,...,0.000000,0.000000,0.000826,0.000000,0.003097,0.000826,0.000000,0.000000,0.000413,0.009703
4,4672,and,0.030822,0.007063,0.014127,0.080479,0.001284,0.008990,0.035317,0.084974,...,0.000000,0.000000,0.000856,0.000214,0.000428,0.001070,0.000428,0.000214,0.001712,0.001712


## Add-one/Laplace Smoothing
- Pretend __we saw each word one more time than we did__!
$ P (w_i|w_{i-1}) = \frac{c(w_{i-1},w_i) + 1}{c(w_i)+|V|} $
- This	__moves the probability mass__ from ‘the rich’ towards ‘the poor’
- Why is it __bad__? 
    - Sometimes ~90% of the probability mass is spread across unseen events.
    - We should know V beforehand!

## Add-b Smoothing
$ P (w_i|w_{i-1}) = \frac{c(w_{i-1},w_i) + \beta}{c(w_i)+\beta|V|} $

__Exercise__: Take again the same __training data__. This time, we use a __bigram LM with Laplace smoothing__:

[s] I am Sam [/s] <br>
[s] Sam I am [/s] <br>
[s] Sam I like [/s] <br>
[s] Sam I do like [/s] <br>
[s] do I like Sam [/s] <br>

1. Give the following __bigram probabilities__ estimated by this model: <br>
P(do|[s]) P(do|Sam) P(Sam|[s]) P(Sam|do) <br>
P(I|Sam) P(I|do) P(like|I) <br>

2. Calculate the __probabilities of the following sequences__ according to this model: <br>
(8) [s] do Sam I like<br>
(9) [s] Sam do I like<br>
Which of the two sequences is more probable according to our LM?

## Good Turing

### Intuition:
You are fishing and caught: __10 carp, 3 perch, 2 whitefish, 1 trout, 1 salmon, 1 eel = 18 fish__ <br>
- How likely is it that __next species is trout__? - 1/18
- How likely is it that __next species is new__ (i.e. catfish or bass)?
- Let’s use our estimate of __things, we saw once to estimate the new things__. - 3/18 (because N_1=3) 
- Now, how likely is it that next species is trout?  - Must be less than 1/18  How to estimate?  
 
### Good-Turing Computation:
- Define __N_c__ as	the __number of N-grams that occur c times__.
- P([unseen]) = N1/N
- Re-estimate counts of words: <br>
$ c* = \frac{(c+1)N_{c+1}}{N_c}$

### Example:
[s] I am Sam [/s] <br>
[s] Sam I am [/s] <br>
[s] Sam I like [/s] <br>
[s] Sam I do like [/s] <br>

- Unseen: 1/5 <br>
- Seen once: c\*(do) = (1+1)\*2/5 = 4/5 <br>

### Problems
- When word occurs c times, but __no word occurs c+1 times__;
- What happens when c(amazing|weather)=0 and c(amazing|trout)=0, and we __smooth unseen bigrams__? These will appear equal, but we would expect the __second to be more likely as weather a more frequent word__. Then, combine uni-gram, bi-gram and tri-gram models.

## Backoff and Interpolation 
- Backoff: use trigram if you have a good evidence, otherwise - bigram, otherwise - unigram!
- Interpola1on: mix unigram, bigram, trigram

### Simple approach:
$P(w_i|w_{i-1}, w_{i-2}) = \lambda_1 P(w_i|w_{i-1},w_{i-2}) + \lambda_2 P(w_i|w_{i-1}) + \lambda_3 P(w_i) $ <br>
$ \lambda_1 + \lambda_2 + \lambda_3 = 1 $
- Estimate lambdas on test set!
- Can have $\lambda$ depending on the context. For example, Witten-Bell Smoothing: <br>
$ \lambda_{w_{i-1}} = \frac{u(w_{i-1})}{u(w_{i-1})+c(w_{i-1})} $ <br>
where :
    - u() - unique counts, c() - raw counts
    - Then: 
$P(w_i|w_{i-1}) = \lambda_{w_{i-1}} P(w_i|w_{i-1},w_{i-2}) + (1-\lambda_{w_{i-1}}) P(w_i) $ <br>

__Exercise__: 
1. __Train an uni-gram, bi-gram and tri-gram models__ on some subset of documents from nltk: (see what's available here: https://www.nltk.org/book/ch02.html)
2. __Test__ the models, computing their __perplexity__, on some data from __same and from a different distribution__.
3. Add and compare __add-one and backoff-and-interpolation for smoothing__.